In [1]:
# TODO: will change to use "pip install -e ." for developing the package
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import time
import re

import pandas as pd
from collections import OrderedDict

import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from deck_crawler.parse_deck import parse_events_from_official
from deck_crawler.parse_deck import reassign_category
from deck_crawler.utils.translator import translate_deck

In [4]:
# flags and parameters
RUN_PARSE = False
DEBUG_PARSE = False

NUM_RESULT_PAGE = 5
NUM_EVENT_PAGE = 1000
NUM_DECK_PAGE = 2

START_DATE = "2023年02月04日"  # include
END_DATE = "2023年02月07日"  # include

TARGET_CATEGORY = "LTB"

In [5]:
# create some folders
EXCEL_FOLDER = "excel"
DB_FOLDER = "deck_db"
folders = [EXCEL_FOLDER, DB_FOLDER]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

if not os.path.exists(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}"):
    os.makedirs(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}")

In [6]:
# loading db
decks = {}
store_file_name = "deck_db/test.json"
if not DEBUG_PARSE:
    store_file_name = "deck_db/store.json"
    if os.path.exists(store_file_name):
        with open(store_file_name, "r") as f:
            decks = json.load(f)

decks = reassign_category(decks)
print("categories in previous result:")
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])
print("\n")
print("number of decks in the previous result:")
print(len(store_code_list))

categories in previous result:
dict_keys(['LOST_ギラティナVSTAR', 'Other_Lost', 'クロススイッチャー', 'LTB_カイオーガ', 'LTB_ヤミラミ_リザードン', 'アルセウスVSTAR', 'アル_ギラティナVSTAR', 'LTB', 'オリジンパルキアVSTAR', 'LTB_空の封印石_other', 'LTB_空の封印石_水超鋼闘', 'れんげきウーラオスVMAX', 'LTB_空の封印石_超闘', 'LTB_空の封印石_水超鋼雷', 'LTB_カイリュー', 'LTB_空の封印石_水超鋼闘雷', 'LTB_空の封印石_水超闘雷', 'LTB_空の封印石_超鋼雷', 'LTB_空の封印石_草超鋼雷', 'LTB_空の封印石_水超雷', 'はくばバドレックスVMAX', 'キュレムVMAX', 'レジエレキVMAX', 'ミライドンex', 'ミュウVMAX', 'ダークライVSTAR', 'かがやくムゲンダイナ', 'そらをとぶピカチュウVMAX', 'プテラVSTAR', 'オリジンディアルガVSTAR', 'アル_レジドラゴVSTAR', 'アル_ジュラルドン', 'こくばバドレックスVMAX', 'アル_そらをとぶピカチュウ', 'アローラロコンVSTAR', 'アルセウス裏工作', '純アルセウスVSTAR', 'レックウザVMAX', 'アル_はくばバドレックスVMAX', 'ミュウツーV-UNION', 'サーナイトex', 'ルギアVSTAR', 'ロトムVSTAR', 'ガラル マタドガス', 'ヒスイ ゾロアークVSTAR', 'ルナトーン', 'ハピナスV', 'ムゲンダイナVMAX', 'ヒスイ ダイケンキVSTAR', 'others', 'レジドラゴVSTAR', 'Other_ギラティナVSTAR', 'パフュートンex', 'ゾロア_ウインディ', 'レジ', 'ゾロア'])


number of decks in the previous result:
1873


In [7]:
# parse events
t1 = time.time()
if RUN_PARSE or DEBUG_PARSE:
    parse_events_from_official(
        decks,
        store_code_list,
        result_page_limit=NUM_RESULT_PAGE,
        event_page_limit=NUM_EVENT_PAGE,
        deck_page_limit=NUM_DECK_PAGE,
    )
t2 = time.time()

print()
print(f"{t2 - t1} seconds")


7.033348083496094e-05 seconds


In [8]:
# save to json
with open(store_file_name, "w") as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [9]:
# show all categories we have
total = 0
for k in decks.keys():
    print(f"[{k}]:\n{len(decks[k])}\n")
    total += len(decks[k])

print(total)

[LOST_ギラティナVSTAR]:
151

[Other_Lost]:
309

[クロススイッチャー]:
101

[LTB_カイオーガ]:
41

[LTB_ヤミラミ_リザードン]:
55

[アルセウスVSTAR]:
161

[アル_ギラティナVSTAR]:
24

[LTB]:
37

[オリジンパルキアVSTAR]:
50

[LTB_空の封印石_other]:
5

[LTB_空の封印石_水超鋼闘]:
2

[れんげきウーラオスVMAX]:
18

[LTB_空の封印石_超闘]:
1

[LTB_空の封印石_水超鋼雷]:
14

[LTB_カイリュー]:
19

[LTB_空の封印石_水超鋼闘雷]:
12

[LTB_空の封印石_水超闘雷]:
6

[LTB_空の封印石_超鋼雷]:
2

[LTB_空の封印石_草超鋼雷]:
2

[LTB_空の封印石_水超雷]:
3

[はくばバドレックスVMAX]:
14

[キュレムVMAX]:
22

[レジエレキVMAX]:
50

[ミライドンex]:
43

[ミュウVMAX]:
112

[ダークライVSTAR]:
8

[かがやくムゲンダイナ]:
18

[そらをとぶピカチュウVMAX]:
73

[プテラVSTAR]:
8

[オリジンディアルガVSTAR]:
4

[アル_レジドラゴVSTAR]:
2

[アル_ジュラルドン]:
37

[こくばバドレックスVMAX]:
4

[アル_そらをとぶピカチュウ]:
55

[アローラロコンVSTAR]:
11

[アルセウス裏工作]:
5

[純アルセウスVSTAR]:
8

[レックウザVMAX]:
2

[アル_はくばバドレックスVMAX]:
3

[ミュウツーV-UNION]:
8

[サーナイトex]:
16

[ルギアVSTAR]:
181

[ロトムVSTAR]:
2

[ガラル マタドガス]:
46

[ヒスイ ゾロアークVSTAR]:
17

[ルナトーン]:
6

[ハピナスV]:
18

[ムゲンダイナVMAX]:
35

[ヒスイ ダイケンキVSTAR]:
1

[others]:
11

[レジドラゴVSTAR]:
2

[Other_ギラティナVSTAR]:
2

[パフュートンex]:
3

[ゾロア_ウインディ]:
4


In [10]:
# show categories we have in the period
number_decks = 0
for k in decks.keys():
    deck_cnt = 0
    for deck in decks[k]:
        if deck["date"] >= START_DATE and deck["date"] <= END_DATE:
            deck_cnt += 1

    print(f"{k}\t{deck_cnt}\n")
    if k == TARGET_CATEGORY:
        number_decks = deck_cnt

LOST_ギラティナVSTAR	113

Other_Lost	172

クロススイッチャー	65

LTB_カイオーガ	13

LTB_ヤミラミ_リザードン	48

アルセウスVSTAR	45

アル_ギラティナVSTAR	20

LTB	29

オリジンパルキアVSTAR	12

LTB_空の封印石_other	4

LTB_空の封印石_水超鋼闘	2

れんげきウーラオスVMAX	11

LTB_空の封印石_超闘	1

LTB_空の封印石_水超鋼雷	0

LTB_カイリュー	0

LTB_空の封印石_水超鋼闘雷	0

LTB_空の封印石_水超闘雷	0

LTB_空の封印石_超鋼雷	0

LTB_空の封印石_草超鋼雷	0

LTB_空の封印石_水超雷	0

はくばバドレックスVMAX	10

キュレムVMAX	0

レジエレキVMAX	33

ミライドンex	37

ミュウVMAX	38

ダークライVSTAR	4

かがやくムゲンダイナ	2

そらをとぶピカチュウVMAX	2

プテラVSTAR	1

オリジンディアルガVSTAR	1

アル_レジドラゴVSTAR	0

アル_ジュラルドン	1

こくばバドレックスVMAX	1

アル_そらをとぶピカチュウ	1

アローラロコンVSTAR	0

アルセウス裏工作	0

純アルセウスVSTAR	3

レックウザVMAX	2

アル_はくばバドレックスVMAX	2

ミュウツーV-UNION	5

サーナイトex	14

ルギアVSTAR	16

ロトムVSTAR	1

ガラル マタドガス	1

ヒスイ ゾロアークVSTAR	9

ルナトーン	2

ハピナスV	0

ムゲンダイナVMAX	0

ヒスイ ダイケンキVSTAR	0

others	2

レジドラゴVSTAR	1

Other_ギラティナVSTAR	1

パフュートンex	3

ゾロア_ウインディ	1

レジ	0

ゾロア	0



In [11]:
# number of decks for the target category
print(number_decks)
if number_decks <= 0:
    raise Exception(f"No decks for {TARGET_CATEGORY} during {START_DATE} to {END_DATE}")

29


In [12]:
# Analysis
df_list = []
common_cols = ["date", "prefecture", "num_players", "rank"]
int_cols = ["num_players", "rank"]

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[TARGET_CATEGORY]):
        deck = translate_deck(deck)

        deck_code = deck["deck_code"]  # row id
        if deck["date"] < START_DATE or deck["date"] > END_DATE:
            continue
        
        pokecard = OrderedDict()
        for col in common_cols:
            pokecard[col] = deck.get(col, "")
        pokecard.update(deck[card_type])

        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.fillna(0)

    # sort rows by date
    df = df.sort_values(by=["date"], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis="rows", numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis="rows")
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks

    # insert rows in df
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    target_rows = [num_rows - 1, num_rows - 2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]

    # sort cols by pick rate
    df = df.sort_values("pick_rate", axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)
    df = df[common_cols + col_list]

    # format data type
    for col in int_cols:
        df[col] = df[col].astype("Int64")

    # store
    df_list.append(df)

In [13]:
# excel writer
writer = pd.ExcelWriter(
    f"{EXCEL_FOLDER}/{TARGET_CATEGORY}/{TARGET_CATEGORY}-{START_DATE}-{END_DATE}.xlsx",
    engine="xlsxwriter",
    mode="w",
)
workbook = writer.book

for sheet_id, sheet_name in enumerate(
    ["pokemons", "tools", "supporters", "stadiums", "energies"]
):
    df_list[sheet_id].to_excel(writer, sheet_name=sheet_name, float_format="%.2f")

    # Make deck_link become a hyperlink
    # Get the xlsxwriter workbook and worksheet objects
    worksheet = writer.sheets[sheet_name]
    for i, deck_id in enumerate(df_list[sheet_id].index):
        if deck_id in ["avg_num_used", "pick_rate"]:
            continue

        # Calculate the row number
        row = i + 2

        # Write the hyperlink to the cell
        worksheet.write_url(
            f"A{row}",
            f"https://www.pokemon-card.com/deck/confirm.html/deckID/{deck_id}",
            string=f"{deck_id}",
        )

    # formatting
    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "top"}
    )
    for col_num, value in enumerate(df_list[sheet_id].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
    worksheet.autofit()

workbook.close()
writer.save()

In [14]:
df_list[0]

,date,prefecture,num_players,rank,花療環環\nS11 049/100,勾魂眼\nS11 044/100,古月鳥\nS11 033/100,瑪納霏\nS12a 029/172,光輝甲賀忍蛙\nS12a 033/172,光輝噴火龍\nS10b 011/071,摔角鷹人\nSV1S 045/078,蓋歐卡\nS8a 007/028,ガラル ファイヤー\nS7D 026/067,ザマゼンタ\nS11a 053/068,卡比獸\nS10a 058/071,怨影娃娃\nSV1V 040/078,詛咒娃娃\nS11 046/100,ガラル ファイヤー\nS12a 079/172,ガラル ファイヤー\nS12a 190/172
pick_rate,NaN,NaN,<NA>,<NA>,1.0,1.000000,1.000000,1.000000,0.448276,0.448276,0.37931,0.241379,0.172414,0.103448,0.103448,0.068966,0.068966,0.068966,0.068966
avg_num_used,NaN,NaN,<NA>,<NA>,4.0,2.551724,2.206897,1.275862,1.000000,1.000000,1.00000,1.000000,1.600000,1.333333,1.000000,2.000000,2.000000,1.000000,1.000000
1V5FFf-3hrzK5-b5kFfk,2023年02月06日(月),東京都,70,4,4.0,2.000000,2.000000,2.000000,1.000000,0.000000,0.00000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
22SSpM-Y40YnJ-XRypy3,2023年02月06日(月),岩手県,32,8,4.0,3.000000,2.000000,2.000000,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FvkdvF-fB11D5-VkFffk,2023年02月06日(月),東京都,32,8,4.0,3.000000,3.000000,1.000000,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ngQgnL-2uL4ci-nnngNQ,2023年02月06日(月),東京都,32,5,4.0,2.000000,2.000000,2.000000,1.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
nQgLNL-4bSKqq-NgNgng,2023年02月06日(月),東京都,32,6,4.0,3.000000,2.000000,1.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
R2ypUp-nyQnPx-XpMpXy,2023年02月06日(月),東京都,32,3,4.0,2.000000,2.000000,1.000000,1.000000,0.000000,0.00000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1VFkff-NiwB04-fkvFFV,2023年02月05日(日),愛知県,64,6,4.0,3.000000,2.000000,1.000000,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FFkvkV-OOaqRe-FvfdV5,2023年02月05日(日),埼玉県,64,15,4.0,3.000000,2.000000,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000


In [15]:
df_list[1]

,date,prefecture,num_players,rank,交替推車,離洞繩,山谷回音喇叭,洗翠的沉重球,巢穴球,寶可裝置3.0,霧之水晶,對戰VIP參加證,放逐吸塵器,交錯切換機,幻想門,能量回收器,寶可夢交替,等級球,講究腰帶,朋友手冊,消除香水,不服輸頭帶,高級球
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,1.000000,0.965517,0.931034,0.896552,0.896552,0.758621,0.724138,0.689655,0.551724,0.551724,0.413793,0.344828,0.241379,0.172414,0.137931,0.103448,0.103448,0.103448
avg_num_used,NaN,NaN,<NA>,<NA>,3.896552,3.965517,1.357143,1.037037,3.153846,2.730769,3.000000,3.952381,1.750000,3.562500,3.062500,2.416667,2.100000,1.571429,1.400000,1.500000,1.333333,1.000000,1.000000
1V5FFf-3hrzK5-b5kFfk,2023年02月06日(月),東京都,70,4,4.000000,4.000000,0.000000,1.000000,3.000000,3.000000,2.000000,4.000000,0.000000,0.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
22SSpM-Y40YnJ-XRypy3,2023年02月06日(月),岩手県,32,8,4.000000,4.000000,2.000000,1.000000,4.000000,2.000000,3.000000,0.000000,2.000000,4.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FvkdvF-fB11D5-VkFffk,2023年02月06日(月),東京都,32,8,4.000000,4.000000,1.000000,1.000000,4.000000,2.000000,3.000000,0.000000,2.000000,4.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
ngQgnL-2uL4ci-nnngNQ,2023年02月06日(月),東京都,32,5,3.000000,4.000000,1.000000,1.000000,2.000000,2.000000,0.000000,4.000000,1.000000,0.000000,4.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
nQgLNL-4bSKqq-NgNgng,2023年02月06日(月),東京都,32,6,4.000000,4.000000,2.000000,0.000000,4.000000,4.000000,4.000000,0.000000,4.000000,0.000000,0.000000,0.000000,4.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000
R2ypUp-nyQnPx-XpMpXy,2023年02月06日(月),東京都,32,3,4.000000,4.000000,1.000000,1.000000,3.000000,0.000000,2.000000,4.000000,1.000000,0.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1VFkff-NiwB04-fkvFFV,2023年02月05日(日),愛知県,64,6,4.000000,4.000000,2.000000,1.000000,3.000000,2.000000,3.000000,4.000000,2.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FFkvkV-OOaqRe-FvfdV5,2023年02月05日(日),埼玉県,64,15,4.000000,4.000000,2.000000,1.000000,2.000000,0.000000,4.000000,4.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
df_list[2]

,date,prefecture,num_players,rank,阿克羅瑪的實驗,克拉拉,杜娟,奇巴納,老大的指令,米莫莎,牡丹,ヒガナの決意
pick_rate,NaN,NaN,<NA>,<NA>,1.0,1.000000,0.793103,0.586207,0.551724,0.068966,0.034483,0.034483
avg_num_used,NaN,NaN,<NA>,<NA>,4.0,2.172414,1.347826,1.000000,1.187500,1.500000,1.000000,1.000000
1V5FFf-3hrzK5-b5kFfk,2023年02月06日(月),東京都,70,4,4.0,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000
22SSpM-Y40YnJ-XRypy3,2023年02月06日(月),岩手県,32,8,4.0,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
FvkdvF-fB11D5-VkFffk,2023年02月06日(月),東京都,32,8,4.0,3.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000
ngQgnL-2uL4ci-nnngNQ,2023年02月06日(月),東京都,32,5,4.0,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
nQgLNL-4bSKqq-NgNgng,2023年02月06日(月),東京都,32,6,4.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
R2ypUp-nyQnPx-XpMpXy,2023年02月06日(月),東京都,32,3,4.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1VFkff-NiwB04-fkvFFV,2023年02月05日(日),愛知県,64,6,4.0,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
FFkvkV-OOaqRe-FvfdV5,2023年02月05日(日),埼玉県,64,15,4.0,2.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000


In [17]:
df_list[3]

,date,prefecture,num_players,rank,海灘場地,寶可補給站,通頂雪道,マグマの滝壺,放逐市
pick_rate,NaN,NaN,<NA>,<NA>,0.689655,0.379310,0.344828,0.241379,0.034483
avg_num_used,NaN,NaN,<NA>,<NA>,1.650000,2.272727,1.400000,1.000000,1.000000
1V5FFf-3hrzK5-b5kFfk,2023年02月06日(月),東京都,70,4,1.000000,0.000000,0.000000,0.000000,0.000000
22SSpM-Y40YnJ-XRypy3,2023年02月06日(月),岩手県,32,8,2.000000,0.000000,1.000000,1.000000,0.000000
FvkdvF-fB11D5-VkFffk,2023年02月06日(月),東京都,32,8,2.000000,0.000000,2.000000,0.000000,0.000000
ngQgnL-2uL4ci-nnngNQ,2023年02月06日(月),東京都,32,5,0.000000,1.000000,0.000000,0.000000,0.000000
nQgLNL-4bSKqq-NgNgng,2023年02月06日(月),東京都,32,6,0.000000,4.000000,0.000000,0.000000,0.000000
R2ypUp-nyQnPx-XpMpXy,2023年02月06日(月),東京都,32,3,1.000000,1.000000,0.000000,0.000000,0.000000
1VFkff-NiwB04-fkvFFV,2023年02月05日(日),愛知県,64,6,2.000000,0.000000,0.000000,1.000000,0.000000
FFkvkV-OOaqRe-FvfdV5,2023年02月05日(日),埼玉県,64,15,1.000000,0.000000,2.000000,0.000000,0.000000


In [18]:
df_list[4]

,date,prefecture,num_players,rank,基本超能量,基本火能量,基本水能量,基本悪エネルギー,基本鋼能量
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,0.448276,0.413793,0.275862,0.103448
avg_num_used,NaN,NaN,<NA>,<NA>,3.413793,2.076923,4.500000,4.125000,3.666667
1V5FFf-3hrzK5-b5kFfk,2023年02月06日(月),東京都,70,4,2.000000,0.000000,4.000000,0.000000,4.000000
22SSpM-Y40YnJ-XRypy3,2023年02月06日(月),岩手県,32,8,4.000000,2.000000,0.000000,0.000000,0.000000
FvkdvF-fB11D5-VkFffk,2023年02月06日(月),東京都,32,8,3.000000,2.000000,0.000000,0.000000,0.000000
ngQgnL-2uL4ci-nnngNQ,2023年02月06日(月),東京都,32,5,3.000000,0.000000,5.000000,4.000000,0.000000
nQgLNL-4bSKqq-NgNgng,2023年02月06日(月),東京都,32,6,4.000000,0.000000,0.000000,0.000000,0.000000
R2ypUp-nyQnPx-XpMpXy,2023年02月06日(月),東京都,32,3,4.000000,0.000000,5.000000,0.000000,3.000000
1VFkff-NiwB04-fkvFFV,2023年02月05日(日),愛知県,64,6,4.000000,2.000000,0.000000,0.000000,0.000000
FFkvkV-OOaqRe-FvfdV5,2023年02月05日(日),埼玉県,64,15,5.000000,2.000000,0.000000,0.000000,0.000000


In [19]:
len(decks["others"])

11